# Рекомендация тарифов

В нашем распоряжении данные о поведении клиентов, которые уже перешли на новый тариф. 

Нам нужно построить модель для задачи классификации, которая выберет подходящий тариф.

## 1. Изучение файлов

Импортируем необходимые библиотеки, и загрузим файл с данными и изучим имеющиеся данные.

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
pd.set_option('mode.chained_assignment', None)
import warnings
warnings.filterwarnings('ignore')

In [2]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


В таблице имеется 5 колонок:
- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

Данные уже готовы для работы, поэтому сразу приступаем к этому этапу.

## 2. Разбейте данные на выборки

Разобьем весь датафрейм на 2 части, на 60% и 40%, затем разобьем 40% пополам, так как нам необходимо, чтобы у нас было 3 выборки, а их соотношение должно быть 3:1:1.

In [3]:
df_train, df_valid = train_test_split(df, test_size=0.40, random_state=12345) 
# random_state ставим любое одинаковое число, для получения одинаковых результатов
df_valid, df_test = train_test_split(df_valid, test_size=0.50, random_state=12345)

Теперь необходимо подготовить переменные с обычными признаками и целевыми признаками.

In [4]:
features_train = df_train.drop('is_ultra', axis=1)
target_train = df_train['is_ultra']

features_valid = df_valid.drop('is_ultra', axis=1)
target_valid = df_valid['is_ultra']

features_test = df_test.drop('is_ultra', axis=1)
target_test = df_test['is_ultra']

Мы подготовили необходимые "заготовки" для исследования, теперь можно перейти непосредственно к самому исследованию.

## 3. Исследуйте модели

Исследовать мы будем 3 модели: дерево решений, случайный лес и логистическая регрессия. 

* Первую модель для исследования возьмем дерево решений.

In [5]:
best_result = 0
best_depth = 0
for depth in range(1, 10):
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_tree.fit(features_train, target_train) # обучение модели на тренировочной выборки
    predictions_tree = model_tree.predict(features_valid) # предсказания модели
    result_tree = accuracy_score(target_valid, predictions_tree) # посчетаем качество модели на валидационной выборке
    if result_tree > best_result:
        best_result = result_tree
        max_depth = depth
        
print("Accuracy наилучшей модели дерева решений на валидационной выборке: {:.3f}".format(best_result))
print('Глубина дерева:', max_depth)

Accuracy наилучшей модели дерева решений на валидационной выборке: 0.785
Глубина дерева: 3


* Вторая модель для исследования будет случайный лес

In [10]:
best_result = 0
best_est = 0
best_depth = 0
for est in range(10, 50):
    model_forest = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
    model_forest.fit(features_train, target_train) # обучение модели на тренировочной выборки
    predictions_forest = model_forest.predict(features_valid) # предсказания модели
    result_forest = accuracy_score(target_valid, predictions_forest)
    if result_forest > best_result:
        best_result = result_forest # наилучшее значение метрики accuracy на валидационных данных
        best_est = est
        best_depth = depth

print("Accuracy наилучшей модели случайного леса на валидационной выборке: {:.3f}".format(best_result))
print('Количество деревьев:', best_est)
print('Максимальная глубина:', best_depth)

Accuracy наилучшей модели случайного леса на валидационной выборке: 0.798
Количество деревьев: 44
Максимальная глубина: 9


* Третья модель логистическая регрессия.

In [7]:
model_log = LogisticRegression(random_state=12345)
model_log.fit(features_train, target_train)
predictions_log = model_log.predict(features_valid)
result_log = accuracy_score(target_valid, predictions_log)

print("Accuracy наилучшей модели логистической регрессии на валидационной выборке: {:.3f}".format(result_log))

Accuracy наилучшей модели логистической регрессии на валидационной выборке: 0.711


Самый высокий показатель точности показала модель случайного леса (accuracy=0,7978227), ее мы возьмем для дальнейшего исследования

## 4. Проверьте модель на тестовой выборке

Подготовим целевой признак и остальные признаки.

In [8]:
features_test = df_test.drop('is_ultra', axis=1)
target_test = df_test['is_ultra']

Теперь мы можем проверить нашу модель на тестовой выборки, с наилучшими гиперпараметрами, которые мы нашли ранее.

In [9]:
model_forest = RandomForestClassifier(random_state=12345, n_estimators=44, max_depth=9)
model_forest.fit(features_train, target_train) # обучение модели на тренировочной выборки
predictions_forest = model_forest.predict(features_test) # предсказания модели
result_forest = accuracy_score(target_test, predictions_forest)
print("Accuracy наилучшей модели на случайном лесе на тестовой выборке: {:.3f}".format(result_forest))

Accuracy наилучшей модели на случайном лесе на тестовой выборке: 0.802


## Вывод.

Нами были полученны данные клиентов, которые уже перешли на новый тариф. Так как данные были представлены в одной таблице, нам необходимо было разделить их на 3 части, тренировочную, валидационную и тестовую в соотношении 3:1:1.Проведя исследование моделей, дерево решений, случайный лес и логистическу регрессию, мы выяснили, что случайный лес показал более точный результат, чем остальные модели. Также мы нашли оптимальные гиперпараметры, для получения высоких значений точности. На валидационной выборке случайный лес показал значение равное 0,798, а на тестовой выборке значение равное 0,802.